# 载入包

In [117]:
import ee
import os
if __name__ == '__main__':
    os.environ['HTTP_PROXY'] = 'http://127.0.0.1:4780'
    os.environ['HTTPS_PROXY'] = 'https://127.0.0.1:4780'
ee.Initialize()

# 导入era5数据

In [127]:
era5_2mt = ee.ImageCollection('ECMWF/ERA5/DAILY').select('maximum_2m_air_temperature').filter(ee.Filter.date('1990-01-01', '2019-12-31'));
#print(era5_2mt);

# 取某一个点实验

In [128]:
u_lon = 4.8148
u_lat = 45.7758
u_poi = ee.Geometry.Point(u_lon, u_lat)

In [129]:
scale = 1000
era5_2mt_poi = era5_2mt.getRegion(u_poi, scale).getInfo()
era5_2mt_poi[:5]

[['id', 'longitude', 'latitude', 'time', 'maximum_2m_air_temperature'],
 ['19900101',
  4.810478346460038,
  45.77365530231022,
  631152000000,
  275.54052734375],
 ['19900102',
  4.810478346460038,
  45.77365530231022,
  631238400000,
  278.7590026855469],
 ['19900103',
  4.810478346460038,
  45.77365530231022,
  631324800000,
  275.4217529296875],
 ['19900104',
  4.810478346460038,
  45.77365530231022,
  631411200000,
  276.40771484375]]

In [140]:
def getdata(lon,lat):
    poi = ee.Geometry.Point(lon, lat)
    scale = 1000
    return era5_2mt.getRegion(poi, scale).getInfo()

In [150]:
len(getdata(-180,-85))

21913

# 定义函数将数据转为时间序列

In [130]:
import pandas as pd

def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    #df = df[['longitude', 'latitude','time','datetime',  *list_of_bands]]
    df = df[['datetime',  *list_of_bands]]

    return df

# 定义函数转化摄氏温度

In [131]:
def t_modis_to_celsius(t_modis):
    """Converts MODIS LST units to degrees Celsius."""
    t_celsius =  t_modis-273
    return t_celsius

In [132]:
era5_2mt_max = ee_array_to_df(era5_2mt_poi,['maximum_2m_air_temperature'])
era5_2mt_max['maximum_2m_air_temperature'] = era5_2mt_max['maximum_2m_air_temperature'].apply(t_modis_to_celsius)
era5_2mt_max.head()

,datetime,maximum_2m_air_temperature
0,1990-01-01,2.540527
1,1990-01-02,5.759003
2,1990-01-03,2.421753
3,1990-01-04,3.407715
4,1990-01-05,3.662720


# 绘图展示

In [104]:
'''import matplotlib.pyplot as plt
import numpy as np
from scipy import optimize
%matplotlib inline

# Fitting curves.
## First, extract x values 
x_data = np.asanyarray(era5_2mt_max['datetime'].apply(date))

## Secondly, extract y values 
y_data = np.asanyarray(era5_2mt_max['maximum_2m_air_temperature'].apply(float))

plt.plot(x_data, y_data,label='curve', color='black', lw=2.5)

# Add some parameters.
ax.set_title('maximum_2m_air_temperature', fontsize=16)
ax.set_xlabel('Date', fontsize=14)
ax.set_ylabel('Temperature [C]', fontsize=14)
ax.set_ylim(-0, 40)
ax.grid(lw=0.2)
ax.legend(fontsize=14, loc='lower right')

plt.show()'''

"import matplotlib.pyplot as plt\nimport numpy as np\nfrom scipy import optimize\n%matplotlib inline\n\n# Fitting curves.\n## First, extract x values \nx_data = np.asanyarray(era5_2mt_max['datetime'].apply(date))\n\n## Secondly, extract y values \ny_data = np.asanyarray(era5_2mt_max['maximum_2m_air_temperature'].apply(float))\n\nplt.plot(x_data, y_data,label='curve', color='black', lw=2.5)\n\n# Add some parameters.\nax.set_title('maximum_2m_air_temperature', fontsize=16)\nax.set_xlabel('Date', fontsize=14)\nax.set_ylabel('Temperature [C]', fontsize=14)\nax.set_ylim(-0, 40)\nax.grid(lw=0.2)\nax.legend(fontsize=14, loc='lower right')\n\nplt.show()"

# 计算每天的热浪阈值

In [99]:
#get_time_window(,,31)
#era5_2mt_max['datetime'].dt.year.head()  #获取年份
#era5_2mt_max['datetime'].dt.month.head() #获取月份
#era5_2mt_max['datetime'].dt.dayofyear.head() #获取日期

In [133]:
import numpy as np
import multiprocessing
import pandas as pd
import datetime

def get_quantile(data,day,win_size,q):
    plusminus = win_size // 2
    delta=datetime.timedelta(days=plusminus)
    start = day-delta
    end = day+delta
    quantile_value = data['maximum_2m_air_temperature'].where((data['datetime']<=end)&(data['datetime']>=start)).quantile(q)
    return quantile_value
    

def HWMId(data, quantile, win_size, n_days, ref_period):
    l = data.shape[0]
    M = np.zeros((l,1))   
    for i in range(l):
        T30y25p = get_quantile(data,data.loc[i,'datetime'],win_size,0.25)
        T30y75p = get_quantile(data,data.loc[i,'datetime'],win_size,0.75)
        M[i] = (data.loc[i,'maximum_2m_air_temperature'] - T30y25p) / (T30y75p - T30y25p)
    M[M<0] = 0
    return M


In [100]:
#hwmid = HWMId(era5_2mt_max,0.9,31,3,slice('1981', '2010'))
#era5_2mt_max['maximum_2m_air_temperature'].where(era5_2mt_max['datetime']<day+15).quantile(0.9)

In [101]:
#print(hwmid)

In [102]:
#hwmid = np.c_[hwmid,hwmid]
#print(hwmid)

# 经纬度信息对应到国家

In [122]:
from geopy.geocoders import Nominatim

In [123]:
geolocator = Nominatim(user_agent = "geoapiExercises")

In [55]:
'''u_lon_s = str(25.594095)
u_lat_s = str(85.137566)
location = geolocator.reverse(u_lon_s+','+u_lat_s'''

In [57]:
'''address = location.raw['address']
country = address.get('country','')
country_code = address.get('country_code')
print(country)
print(country_code)'''

India
in


In [124]:
def getcountry(lon,lat):
    lon_s = str(lon)
    lat_s = str(lat)
    location = geolocator.reverse(lon_s+','+lat_s)
    address = location.raw['address']
    country = address.get('country','')
    #country_code = address.get('country_code')
    return country

In [152]:
#getcountry(25,85)

# 提取各经纬度点上的数据

In [153]:
from global_land_mask import globe
k = -1;
countrylist = np.zeros((1,360*171))
for i in range(-85,85,1):
    for j in range(-179,179,1):
        is_on_land = globe.is_land(i,j)
        if (is_on_land):
            k = k+1;
            #countrylist[k] = getcountry(i,j)
            era5_2mt_poi = getdata(j,i)
            era5_2mt_max = ee_array_to_df(era5_2mt_poi,['maximum_2m_air_temperature'])
            era5_2mt_max['maximum_2m_air_temperature'] = era5_2mt_max['maximum_2m_air_temperature'].apply(t_modis_to_celsius)
            hwmid = HWMId(era5_2mt_max,0.9,31,3,slice('1990', '2019'))
            '''print(k)
            print(i,' ',j)
            print(era5_2mt_poi[:5])
            print(era5_2mt_max.head())
            print(era5_2mt_max.shape[0])
            print(hwmid)
            print(hwmid_matrix)'''
            if (k==0):
                hwmid_matrix = hwmid
            else:
                hwmid_matrix = np.c_[hwmid_matrix,hwmid]


0
-85   -179
[['id', 'longitude', 'latitude', 'time', 'maximum_2m_air_temperature'], ['19900101', -179.00279509007646, -85.00308375980973, 631152000000, 253.61961364746094], ['19900102', -179.00279509007646, -85.00308375980973, 631238400000, 253.35577392578125], ['19900103', -179.00279509007646, -85.00308375980973, 631324800000, 253.51345825195312], ['19900104', -179.00279509007646, -85.00308375980973, 631411200000, 253.8749542236328]]
0   datetime  maximum_2m_air_temperature
0 1990-01-01                  -19.380386
1 1990-01-02                  -19.644226
2 1990-01-03                  -19.486542
3 1990-01-04                  -19.125046
4 1990-01-05                  -17.912659
10956
[[0.        ]
 [0.        ]
 [0.        ]
 ...
 [0.04508716]
 [0.        ]
 [0.0063765 ]]
[[0.        ]
 [0.        ]
 [0.        ]
 ...
 [0.04508716]
 [0.        ]
 [0.0063765 ]]
1
-85   -178
[['id', 'longitude', 'latitude', 'time', 'maximum_2m_air_temperature'], ['19900101', -177.99668197186259, -85.00308

KeyboardInterrupt: 

In [115]:
hwmid_matrix.shape

(0, 76)

In [107]:
np.savetxt('heatwaves.csv', hwmid_matrix, delimiter = ',')